In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.display import display, Javascript

In [ ]:
# 런타임 오류 방지 함수
# # 이 함수는 Colab의 런타임 연결이 끊기는 것을 방지하기 위해 60초마다 연결 버튼을 자동으로 클릭합니다.
def keep_alive():
    display(Javascript('''
        function ClickConnect(){
            console.log("클릭 연결 버튼");
            document.querySelector("colab-connect-button").click()
        }
        setInterval(ClickConnect, 60000)
    '''))

# 데이터 로드 및 전처리
def load_and_preprocess_data():
    # '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/json to df.csv'에서 CSV 파일을 읽어 데이터프레임으로 로드합니다.
    df = pd.read_csv('/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/json to df.csv')

    # 'info', 'images', 'annotations', 'equipment' 열의 문자열로 저장된 딕셔너리를 실제 딕셔너리로 변환합니다.
    for col in ['info', 'images', 'annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)

    # 'split' 열이 'Training'인 행만 선택합니다.
    df = df[df['split'] == 'Training']

    # preprocess_data 함수를 호출하여 추가적인 전처리를 수행합니다.
    return preprocess_data(df)

# 데이터 전처리
def preprocess_data(df):
    def process_annotations(anno):
        # annotations 열의 데이터를 처리합니다.
        if isinstance(anno, dict):
            # 값이 리스트인 경우에는 해당 리스트의 길이를 반환하고, 리스트가 아닌 경우에는 원래 값을 그대로 반환
            # facepart 0의 경우 'acne' 항목을 개수로 변환
            return {k: len(v) if isinstance(v, list) else v for k, v in anno.items()}
        return {}

    # 각 행에 대해 annotations와 equipment를 처리합니다.
    df['annotations'] = df['annotations'].apply(process_annotations)
    # 'equipment' 열의 딕셔너리 value들 가져오기 딕셔너리가 아니고 None이면 빈 딕셔너리로 반환
    df['equipment'] = df['equipment'].apply(lambda x: x if isinstance(x, dict) else {})
    return df

# 오버샘플링 함수
def oversample_data(X, y):
    # RandomOverSampler를 사용하여 샘플수가 적은 클래스의 샘플을 증가시킵니다.
    ros = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X.to_frame(), y)
    return X_resampled.iloc[:, 0], y_resampled

# 클래스 가중치 계산 함수
def compute_class_weights(y):
    # 클래스별 가중치를 계산합니다.
    classes = np.unique(y)
    weights = compute_class_weight('balanced', classes=classes, y=y)
    return dict(zip(classes, weights))

# 데이터 증강
# 이미지 데이터 증강을 수행합니다.
def augment_data(image):
    image = tf.image.random_flip_up_down(image) # 상하 반전
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2) # 대비 조정
    image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)) # 회전
    return image

# 이미지 로드 및 전처리 함수
def load_and_preprocess_image(image_path):
    # 주어진 경로(image_path)에서 이미지를 파일로 읽어오기
    img = tf.io.read_file(image_path)
    # 읽어온 파일을 JPEG 형식의 이미지로 디코딩합니다. 이미지는 3개의 채널(RGB)을 가집니다.
    img = tf.image.decode_jpeg(img, channels=3)
    # 디코딩된 이미지를 224x224 크기로 리사이즈
    img = tf.image.resize(img, [224, 224])
    # EfficientNet 모델에 맞게 이미지를 전처리합니다. 이 단계는 이미지의 픽셀 값을 모델이 요구하는 형식(예: 정규화 등)으로 변환합니다.
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    # 전처리된 이미지를 반환
    return img

# 데이터 제너레이터를 생성합니다.
# 이 함수는 이미지 경로와 레이블을 받아서 텐서플로우 데이터셋을 생성합니다.
def create_data_generator(X, y, directory, batch_size=32, is_training=True):
    # 이 함수는 데이터를 하나씩 생성하여 반환합니다.
    def gen():
        # X의 길이만큼 반복문을 실행합니다. 즉, 각 이미지 경로에 대해 반복합니다.
        for i in range(len(X)):
            # X에서 현재 인덱스 i에 해당하는 이미지 경로를 가져옵니다.
            img_path = X.iloc[i]
            if os.path.exists(img_path):  # 이미지 파일이 실제로 존재하는지 확인합니다.
                # 이미지 파일이 존재하면, load_and_preprocess_image 함수를 사용하여 이미지를 로드하고 전처리합니다
                img = load_and_preprocess_image(img_path)
                # y에서 현재 인덱스 i에 해당하는 레이블을 가져옵니다.
                label = y.iloc[i]
                # 전처리된 이미지와 레이블을 제너레이터의 출력으로 반환합니다.
                yield img, label
            # 이미지 파일이 존재하지 않을 경우에 대해 처리합니다.
            else:
                print(f"Skipping missing file: {img_path}")  # 누락된 파일 정보 출력
    # tf.data.Dataset.from_generator를 사용하여 제너레이터로부터 텐서플로우 데이터셋을 생성합니다.
    dataset = tf.data.Dataset.from_generator(
        # 데이터셋을 생성할 때 사용할 제너레이터 함수로 gen을 지정합니다.
        gen,
        # 데이터셋의 출력 타입과 형태를 정의
        output_signature=(
            # 이미지 텐서는 (224, 224, 3) 형태의 tf.float32 타입입니다.
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
            # 레이블은 스칼라 tf.float32 타입입니다.
            tf.TensorSpec(shape=(), dtype=tf.float32)
        )
    )
    # is_training이 True일 경우
    if is_training:
        # 데이터셋을 셔플링합니다. 셔플링은 데이터의 순서를 무작위로 섞어줍니다.
        dataset = dataset.shuffle(buffer_size=len(X))
    # 데이터셋을 주어진 batch_size로 배치하고, .prefetch(tf.data.AUTOTUNE)을 사용하여 데이터 로딩과 모델 훈련을 병렬로 수행합니다.
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset # 최종적으로 생성된 데이터셋을 반환합니다

# 검증 데이터 생성기
# is_training이 False여서 셔플링x
def create_val_data_generator(X, y, directory, batch_size=32):
    return create_data_generator(X, y, directory, batch_size, is_training=False)

# 모델 생성 함수
# create_model이라는 함수를 정의합니다. 이 함수는 출력 차원(output_dim)과 모델 유형(model_type)을 인자로 받습니다.
def create_model(output_dim, model_type):
    # 'ImageNet'이라는 큰 데이터셋으로 학습된 가중치를 사용합니다.
    # EfficientNetB0 모델의 마지막 부분에는 이미지가 어떤 종류인지 분류하는 레이어가 있습니다.
    # include_top=False는 이 마지막 분류 레이어를 사용하지 않고 우리는 직접 새로운 분류 레이어를 추가할 것입니다.
    # 모델에 입력되는 이미지의 크기를 (224, 224, 3) -> 3은 색상 채널(RGB)을 의미
    base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # 기본 모델의 출력을 글로벌 평균 풀링 레이어를 통해 처리합니다. 이는 각 채널의 평균 값을 구합니다.
    # 각 특성 맵의 평균값을 계산하여 전체적인 특성을 포착
    # 색소침착의 전반적인 분포나 주름의 평균적인 정도를 감지하는 데 유용
    x_gap = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

    # 기본 모델의 출력을 글로벌 맥스 풀링 레이어를 통해 처리합니다. 이는 각 채널의 최댓값을 구합니다.
    # 각 특성 맵의 최대값을 선택하여 가장 두드러진 특성을 포착
    # 가장 깊은 주름이나 가장 심한 색소침착 부위를 감지하는 데 효과적
    x_gmp = tf.keras.layers.GlobalMaxPooling2D()(base_model.output)

    # 글로벌 평균 풀링과 글로벌 맥스 풀링의 출력을 병합합니다.
    x = tf.keras.layers.Concatenate()([x_gap, x_gmp])
    # 병합된 출력에 Dense 레이어를 추가하고, 256개의 유닛(뉴런)과 ReLU 활성화 함수를 사용합니다.
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    # 40%의 노드를 랜덤하게 드롭아웃합니다. 이는 과적합을 방지하기 위함 입니다.
    x = tf.keras.layers.Dropout(0.4)(x)

    if model_type == 'classification': # 분류
        # 분류 모델인 경우, Dense 레이어를 추가하여 output_dim 개수의 유닛(뉴런)과 소프트맥스 활성화 함수를 사용합니다. 이는 다중 클래스 분류를 위함
        output = tf.keras.layers.Dense(output_dim, activation='softmax')(x)
    else:  # 회귀
        # 회귀 모델인 경우, Dense 레이어를 추가하여 단일 유닛(뉴런)을 사용합니다.
        output = tf.keras.layers.Dense(1)(x)
    # 입력이 base_model.input이고 출력이 output인 케라스 모델 객체를 반환합니다.
    return tf.keras.Model(inputs=base_model.input, outputs=output)

# 모델의 학습 과정을 시각화합니다.
# 훈련 및 검증 손실, 정확도(분류) 또는 MAE(회귀)를 시각화합니다.
# history 객체를 이용해 학습 손실(loss)과 평가 지표(metric)의 변화를 그래프로 그립니다.
def plot_performance(history, metric_name, facepart, feature, model_type):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # 손실함수 plot
    ax1.plot(history.history['loss'], label='Train Loss')
    ax1.plot(history.history['val_loss'], label='Validation Loss')
    ax1.set_title(f'{feature} Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()

    # 정확도(분류) 또는 MAE(회귀) plot
    ax2.plot(history.history[metric_name], label=f'Train {metric_name.upper()}')
    ax2.plot(history.history[f'val_{metric_name}'], label=f'Validation {metric_name.upper()}')
    ax2.set_title(f'{feature} {metric_name.upper()}')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel(metric_name.upper())
    ax2.legend()

    plt.tight_layout()
    plt.savefig(f'/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/model/facepart_{facepart}_{feature}_{model_type}_performance.png')
    plt.close()

# 모델 훈련 함수
# 모델을 학습시킵니다.
def train_model(model, train_data, val_data, facepart, feature, model_type, epochs=30, batch_size=32):
    # 초기 학습률을 설정
    initial_lr = 1e-4
    # Adam 옵티마이저를 초기화
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)

    # 회귀 문제의 경우, 손실 함수는 평균 제곱 오차(mean_squared_error)이고, 평가 지표는 평균 절대 오차(mae)
    if model_type == 'regression':
        loss = 'mean_squared_error'
        metric = 'mae'
    # 분류 문제의 경우, 손실 함수는 희소 범주형 교차 엔트로피(sparse_categorical_crossentropy)이고, 평가 지표는 정확도(accuracy)
    else:  # classification
        loss = 'sparse_categorical_crossentropy'
        metric = 'accuracy'
    # 설정된 옵티마이저, 손실 함수, 평가 지표를 사용해서 모델을 컴파일
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    # 모델 체크포인트 콜백을 설정합니다. 이 콜백은 매 5 에포크마다 손실이 최소인 모델을 저장합니다.
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=f'/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/model/facepart_{facepart}_{feature}_{model_type}_checkpoint_{{epoch:02d}}.keras',
        save_best_only=True,
        save_weights_only=False,
        monitor='loss',
        mode='min',
        save_freq=5)

    # 학습 속도 감소 콜백을 설정합니다. 검증 손실(val_loss)이 개선되지 않으면 학습 속도를 줄입니다.
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
    # 조기 종료 콜백을 설정합니다. 검증 손실이 10 에포크 동안 개선되지 않으면 학습을 중지하고, 가장 좋은 모델 가중치를 복원합니다.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    # 모델을 학습
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=epochs,
        verbose=1,
        callbacks=[checkpoint_callback, reduce_lr, early_stopping]
    )
    # 최종 학습된 모델을 저장
    model.save(f'/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/model/facepart_{facepart}_{feature}_{model_type}_final_model.keras')
    return history

# 이미지 경로 가져오기
def get_image_path(info, facepart, train_directory):
    if facepart == 0:
        filename = info['filename']
        path = f'/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Training/01.원천데이터/{filename}'
        return path
    else:
        filename = info['filename'].split('.')[0]
        return str(os.path.join(train_directory, f"{filename}_{facepart}.jpg"))

# facepart별 모델 훈련 함수
# 각 facepart에 대한 모델을 학습시킵니다.
def train_facepart_models(facepart, train_classification=True, train_regression=True):
    print(f"Processing facepart {facepart}")

    # facepart별 데이터프레임 생성
    facepart_df = df[df['images'].apply(lambda x: x['facepart'] == facepart)]
    # 유효한 bbox를 가진 행만 사용
    def valid_bbox(bbox):
        if bbox is None:
            return False
        if isinstance(bbox, list) and len(bbox) == 4:
            if bbox == ['None', 'None', 'None', 'None']:
                return False
            return all(isinstance(b, int) and b >= 0 for b in bbox)
        return False
    # bbox 유효성 검증
    facepart_df = facepart_df[facepart_df['images'].apply(lambda x: valid_bbox(x.get('bbox')))]
    # facepart 1~8 이미지 파일 경로
    train_directory = f'/gdrive/MyDrive/Final project/1_Red/4_데이터탐색_전처리/facepart별 피부 이미지/Training_cropped/{facepart}'
    # 이미지 경로 가져오기
    X = facepart_df['info'].apply(lambda x: get_image_path(x, facepart, train_directory))

    if train_classification: # 분류
        # annotations컬럼의 키마다 진행
        for feature in facepart_df['annotations'].iloc[0].keys(): # facepart별 annotations 키값들 추출
            if feature == 'acne' and facepart == 0: # facepart0일 때
                # y(클래스)는 여드름 개수
                y = facepart_df['annotations'].apply(lambda x: x['acne'])
            else: # 나머지 facepart
                # y(클래스)는 annotations 컬럼의 키(feature) 값들
                y = facepart_df['annotations'].apply(lambda x: x.get(feature, None))
            # 클래스가 1개 초과일 경우 학습
            if y.nunique() > 1:
                print(f"Starting classification training for facepart {facepart}, feature {feature}")
                # 9:1 비율로 분할
                X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
                # 샘플 수가 적은 클래스가 있어서 오버샘플링 적용
                X_train_resampled, y_train_resampled = oversample_data(X_train, y_train)
                # 학습, 검증 데이터 생성기 생성
                train_generator = create_data_generator(X_train_resampled, y_train_resampled, train_directory)
                val_generator = create_val_data_generator(X_val, y_val, train_directory)

                model = create_model(y.nunique(), 'classification') # 분류 모델 생성
                history = train_model(model, train_generator, val_generator, facepart, feature, 'classification') # 모델 학습
                plot_performance(history, 'accuracy', facepart, feature, 'classification') # 학습 성능 시각화

    if train_regression: # 회귀
        # # 회귀 모델 학습에 사용할 피처 리스트
        regression_features = ['forehead_moisture', 'r_cheek_moisture', 'l_cheek_moisture', 'chin_moisture',
                               'chin_elasticity_R2', 'r_cheek_elasticity_R2', 'l_cheek_elasticity_R2',
                               'forehead_elasticity_R2', 'pigmentation_count', 'r_cheek_pore', 'l_cheek_pore']

        for feature in regression_features: # 피처리스트에 있는 피처별 진행
            if feature in facepart_df['equipment'].iloc[0]: # 현재 특징이 데이터프레임에 존재하는지 확인
                y = facepart_df['equipment'].apply(lambda x: x.get(feature, None)) # y 값을 피처로 설정

                if not y.isnull().all(): # y 값이 모두 결측치가 아닌 경우에만 학습
                    print(f"Starting regression training for facepart {facepart}, feature {feature}") # 학습 시작 메시지 출력
                    # 9:1 비율로 분할
                    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
                    # 샘플 수가 적은 클래스가 있어서 오버샘플링 적용
                    X_train_resampled, y_train_resampled = oversample_data(X_train, y_train)
                    # 학습, 검증 데이터 생성기 생성
                    train_generator = create_data_generator(X_train_resampled, y_train_resampled, train_directory)
                    val_generator = create_val_data_generator(X_val, y_val, train_directory)

                    model = create_model(1, 'regression')  # 회귀 모델 생성
                    history = train_model(model, train_generator, val_generator, facepart, feature, 'regression') # 모델 학습
                    plot_performance(history, 'mae', facepart, feature, 'regression') # 학습 성능 시각화

if __name__ == "__main__":
    keep_alive()
    df = load_and_preprocess_data()
    user_input = input("처리할 facepart 범위를 선택하세요 (1: 0-2, 2: 3-6, 3: 7-8): ")
    if user_input == '1':
        facepart_range = [0, 1, 2]
    elif user_input == '2':
        facepart_range = [3, 4, 5, 6]
    elif user_input == '3':
        facepart_range = [7, 8]
    else:
        print("잘못된 입력입니다.")
        exit()
    train_class = input("분류 모델을 학습하시겠습니까? (y/n): ").lower() == 'y'
    train_reg = input("회귀 모델을 학습하시겠습니까? (y/n): ").lower() == 'y'
    for facepart in facepart_range:
        train_facepart_models(facepart, train_classification=train_class, train_regression=train_reg)

<IPython.core.display.Javascript object>

처리할 facepart 범위를 선택하세요 (1: 0-2, 2: 3-6, 3: 7-8): 3
분류 모델을 학습하시겠습니까? (y/n): y
회귀 모델을 학습하시겠습니까? (y/n): n
Processing facepart 7
Starting classification training for facepart 7, feature lip_dryness
16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/30


# 테스트

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os

def load_and_preprocess_data(split='Validation'):
    df = pd.read_csv('/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/json to df.csv')
    for col in ['info', 'images', 'annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)
    df = df[df['split'] == split]
    return df

def get_image_path(info, facepart):
    filename = info['filename'].split('.')[0]
    return f'/gdrive/MyDrive/Final project/1_Red/4_데이터탐색_전처리/facepart별 피부 이미지/Validation_cropped/{facepart}/{filename}_{facepart}.jpg'

# 이미지 로드 및 전처리 함수 (에러 처리 추가)
def load_and_preprocess_image(image_path):
    if not os.path.exists(image_path):
        print(f"파일을 찾을 수 없습니다: {image_path}")
        return None
    try:
        img = tf.io.read_file(image_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [224, 224])
        img = tf.keras.applications.efficientnet.preprocess_input(img)
        return img
    except tf.errors.InvalidArgumentError:
        print(f"이미지를 처리할 수 없습니다: {image_path}")
        return None

# 데이터셋 생성 함수 (에러 처리 추가)
def create_dataset(X, y):
    def generator():
        for image_path, label in zip(X, y):
            img = load_and_preprocess_image(image_path)
            if img is not None:
                yield img, label

    return tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    )

# 성능 평가 및 시각화 함수
def evaluate_model(model, dataset, y, feature, facepart):
    # 예측
    y_true = []
    y_pred = []
    for images, labels in dataset:
        predictions = model.predict(images)
        y_true.extend(labels.numpy())
        y_pred.extend(np.argmax(predictions, axis=1))

    # 분류 리포트 출력
    print(classification_report(y_true, y_pred))

    # 혼동 행렬 시각화
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - Facepart {facepart}, Feature {feature}')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(f'/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/model/facepart_{facepart}_{feature}_confusion_matrix.png')
    plt.close()

# 메인 실행 코드
if __name__ == "__main__":
    facepart = 7
    df = load_and_preprocess_data('Validation')
    # 제거할 파일명 리스트
    filenames_to_remove = ['0615_01_L15.jpg', '0615_01_R30.jpg', '0627_01_Ft.jpg']

    # 조건에 맞는 행을 제거
    df = df[~df['info'].apply(lambda x: x['filename'] in filenames_to_remove)]
    facepart_df = df[df['images'].apply(lambda x: x['facepart'] == facepart)]

    for feature in facepart_df['annotations'].iloc[0].keys():
        print(f"Evaluating feature: {feature}")

        # 데이터 준비
        X = facepart_df['info'].apply(lambda x: get_image_path(x, facepart))
        y = facepart_df['annotations'].apply(lambda x: x.get(feature, None))

        # 유효한 데이터만 선택
        valid_indices = ~y.isnull()
        X = X[valid_indices]
        y = y[valid_indices]

        if y.nunique() > 1:
            # 모델 로드
            model_path = f'/gdrive/MyDrive/Final project/1_Red/5_분석모델링/피부진단/model/facepart_7_classification_checkpoint_01.keras'
            if not os.path.exists(model_path):
                print(f"모델 파일을 찾을 수 없습니다: {model_path}")
                continue
            model = tf.keras.models.load_model(model_path)

            # 데이터셋 생성
            dataset = create_dataset(X.tolist(), y.tolist())
            dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)

            # 모델 평가
            evaluate_model(model, dataset, y, feature, facepart)
        else:
            print(f"Skipping {feature} due to insufficient classes")

# 데이터 전처리

In [ ]:
df = pd.read_csv('/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 원본 데이터/json_to_df_result.csv')
for col in ['annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [ ]:
len(df)

112905

In [ ]:
df['source_type'].value_counts()

source_type
Training      100386
Validation     12519
Name: count, dtype: int64

In [ ]:
base_path = '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 원본 데이터/라벨링 데이터/'
face_list = ['face','forehead','glabellus','l_perocular','r_perocular','l_cheek','r_cheek','lip','chin']
for no, facepart in zip(range(9),face_list):
    new = df[df['facepart']== no]
    new.to_csv(base_path + f'{facepart}_origin.csv')
    new.dropna(inplace=True)
    new.to_csv(base_path + f'{facepart}.csv')
    print(facepart)
    new.info()
    print()

<ipython-input-52-d35697e3a93d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.dropna(inplace=True)


face
<class 'pandas.core.frame.DataFrame'>
Index: 12545 entries, 0 to 112896
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source_type  12545 non-null  object
 1   annotations  12545 non-null  object
 2   equipment    12545 non-null  object
 3   filename     12545 non-null  object
 4   id           12545 non-null  int64 
 5   gender       12545 non-null  object
 6   age          12545 non-null  int64 
 7   date         12545 non-null  object
 8   skin_type    12545 non-null  int64 
 9   sensitive    12545 non-null  int64 
 10  device       12545 non-null  int64 
 11  width        12545 non-null  int64 
 12  height       12545 non-null  int64 
 13  angle        12545 non-null  int64 
 14  facepart     12545 non-null  int64 
 15  bbox         12545 non-null  object
dtypes: int64(9), object(7)
memory usage: 1.6+ MB



<ipython-input-52-d35697e3a93d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.dropna(inplace=True)


forehead
<class 'pandas.core.frame.DataFrame'>
Index: 12504 entries, 1 to 112897
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source_type  12504 non-null  object
 1   annotations  12504 non-null  object
 2   equipment    12504 non-null  object
 3   filename     12504 non-null  object
 4   id           12504 non-null  int64 
 5   gender       12504 non-null  object
 6   age          12504 non-null  int64 
 7   date         12504 non-null  object
 8   skin_type    12504 non-null  int64 
 9   sensitive    12504 non-null  int64 
 10  device       12504 non-null  int64 
 11  width        12504 non-null  int64 
 12  height       12504 non-null  int64 
 13  angle        12504 non-null  int64 
 14  facepart     12504 non-null  int64 
 15  bbox         12504 non-null  object
dtypes: int64(9), object(7)
memory usage: 1.6+ MB



<ipython-input-52-d35697e3a93d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.dropna(inplace=True)


...

r_cheek
<class 'pandas.core.frame.DataFrame'>
Index: 12304 entries, 6 to 112902
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source_type  12304 non-null  object
 1   annotations  12304 non-null  object
 2   equipment    12304 non-null  object
 3   filename     12304 non-null  object
 4   id           12304 non-null  int64 
 5   gender       12304 non-null  object
 6   age          12304 non-null  int64 
 7   date         12304 non-null  object
 8   skin_type    12304 non-null  int64 
 9   sensitive    12304 non-null  int64 
 10  device       12304 non-null  int64 
 11  width        12304 non-null  int64 
 12  height       12304 non-null  int64 
 13  angle        12304 non-null  int64 
 14  facepart     12304 non-null  int64 
 15  bbox         12304 non-null  object
dtypes: int64(9), object(7)
memory usage: 1.6+ MB



<ipython-input-52-d35697e3a93d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.dropna(inplace=True)


lip
<class 'pandas.core.frame.DataFrame'>
Index: 12522 entries, 7 to 112903
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source_type  12522 non-null  object
 1   annotations  12522 non-null  object
 2   equipment    12522 non-null  object
 3   filename     12522 non-null  object
 4   id           12522 non-null  int64 
 5   gender       12522 non-null  object
 6   age          12522 non-null  int64 
 7   date         12522 non-null  object
 8   skin_type    12522 non-null  int64 
 9   sensitive    12522 non-null  int64 
 10  device       12522 non-null  int64 
 11  width        12522 non-null  int64 
 12  height       12522 non-null  int64 
 13  angle        12522 non-null  int64 
 14  facepart     12522 non-null  int64 
 15  bbox         12522 non-null  object
dtypes: int64(9), object(7)
memory usage: 1.6+ MB



<ipython-input-52-d35697e3a93d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.dropna(inplace=True)


chin
<class 'pandas.core.frame.DataFrame'>
Index: 12545 entries, 8 to 112904
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source_type  12545 non-null  object
 1   annotations  12545 non-null  object
 2   equipment    12545 non-null  object
 3   filename     12545 non-null  object
 4   id           12545 non-null  int64 
 5   gender       12545 non-null  object
 6   age          12545 non-null  int64 
 7   date         12545 non-null  object
 8   skin_type    12545 non-null  int64 
 9   sensitive    12545 non-null  int64 
 10  device       12545 non-null  int64 
 11  width        12545 non-null  int64 
 12  height       12545 non-null  int64 
 13  angle        12545 non-null  int64 
 14  facepart     12545 non-null  int64 
 15  bbox         12545 non-null  object
dtypes: int64(9), object(7)
memory usage: 1.6+ MB



In [ ]:
chin_df = pd.read_csv(base_path + 'chin.csv', index_col=0)
for col in ['annotations','equipment']:
        chin_df[col] = chin_df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)
chin_df.head(1)

,source_type,annotations,equipment,filename,id,gender,age,date,skin_type,sensitive,device,width,height,angle,facepart,bbox
8,Validation,{'chin_sagging': 1},"{'chin_moisture': 78.667, 'chin_elasticity_R0'...",0001_01_Fb.jpg,1,F,13,2023-08-17,0,0,0,2136,3216,2,8,"[396, 2671, 1749, 3144]"


In [ ]:
acne_list = []
# wrinkle_list = []
for d in chin_df['annotations']:
    acne_list.append(d['chin_sagging'])
    # wrinkle_list.append(d['r_cheek_pigmentation'])
chin_df['sagging'] = acne_list
# r_cheek_df['pigmentation'] = wrinkle_list
chin_df.head(1)

,source_type,annotations,equipment,filename,id,gender,age,date,skin_type,sensitive,device,width,height,angle,facepart,bbox,sagging
8,Validation,{'chin_sagging': 1},"{'chin_moisture': 78.667, 'chin_elasticity_R0'...",0001_01_Fb.jpg,1,F,13,2023-08-17,0,0,0,2136,3216,2,8,"[396, 2671, 1749, 3144]",1


In [ ]:
chin_df.to_csv(base_path + 'chin.csv')

In [ ]:
face_list = ['face','forehead','glabellus','l_perocular','r_perocular','l_cheek','r_cheek','lip','chin']
base_path = '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 원본 데이터/라벨링 데이터/'
df = pd.read_csv(base_path + 'l_cheek.csv')
df.head(3)

,Unnamed: 0,source_type,annotations,equipment,filename,id,gender,age,date,skin_type,sensitive,device,width,height,angle,facepart,bbox,pore,pigmentation
0,5,Validation,"{'l_cheek_pore': 2, 'l_cheek_pigmentation': 3}","{'l_cheek_moisture': 71.667, 'l_cheek_elastici...",0001_01_Fb.jpg,1,F,13,2023-08-17,0,0,0,2136,3216,2,5,"[327, 1879, 829, 2538]",2,3
1,14,Validation,"{'l_cheek_pore': 2, 'l_cheek_pigmentation': 3}","{'l_cheek_moisture': 71.667, 'l_cheek_elastici...",0001_01_Ft.jpg,1,F,13,2023-08-17,0,0,0,2136,3216,1,5,"[175, 1361, 788, 1964]",2,3
2,23,Validation,"{'l_cheek_pore': 2, 'l_cheek_pigmentation': 3}","{'l_cheek_moisture': 71.667, 'l_cheek_elastici...",0001_01_F.jpg,1,F,13,2023-08-17,0,0,0,2136,3216,0,5,"[257, 1680, 832, 2355]",2,3
